<a href="https://colab.research.google.com/github/Hongbi-Kim/Writing_lyrics/blob/master/kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
## Mecab 설치

!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git 
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab190912.sh
%cd ../

     |████████████████████████████████| 19.4 MB 9.6 MB/s 
     |████████████████████████████████| 448 kB 40.0 MB/s 
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 16.05 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-04-20 05:40:45--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loc

In [4]:
!pip install sentence_transformers

import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [5]:
!pip install bertopic

In [1]:
!pip install bertopic[visualization]

     |████████████████████████████████| 57 kB 4.2 MB/s 
     |████████████████████████████████| 88 kB 6.2 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 5.2 MB 63.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.0 MB 52.7 MB/s 
     |████████████████████████████████| 1.2 MB 49.3 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 6.6 MB 50.7 MB/s 
     |████████████████████████████████| 636 kB 55.5 MB/s 
     |████████████████████████████████| 895 kB 56.7 MB/s 
     |████████████████████████████████| 1.1 MB 56.9 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330859 sha256=3cc74364628369c0a518b723b6c478c86ad00d6941140f9857ccd117f1b7eccc
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef

In [16]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

In [6]:
from konlpy.tag import Mecab
mecab = Mecab()

In [7]:
import pandas as pd
df = pd.read_pickle('/content/drive/MyDrive/5조_파이널PJT/코드/data/melonlyric_resetindex.pkl')
df

,pages,rank,title,lyric,artist,genres,writer
0,1,1,사랑인가 봐,"너와 함께 하고 싶은 일들을,상상하는 게,요즘 내 일상이 되고,너의 즐거워하는 모습...",멜로망스,"발라드, 국내드라마",김민석 (멜로망스)
1,1,2,취중고백,"뭐하고 있었니 늦었지만,잠시 나올래,너의 집 골목에 있는,놀이터에 앉아 있어,친구들...",김민석 (멜로망스),발라드,김희탐
2,1,3,듣고 싶을까,"우리 함께 듣던 그 노랠 듣고 싶을까,듣고 나서 잠시 날 생각은 할까,아주 혹시라도...",MSG워너비(M.O.M),발라드,Dr.JO
3,1,4,사랑은 늘 도망가,"눈물이 난다 이 길을 걸으면,그 사람 손길이 자꾸 생각이 난다,붙잡지 못하고 가슴만...",임영웅,"발라드, 국내드라마",강태규
4,1,5,언제나 사랑해,"둘이 손잡고 걸을 때,마주 보며 또 웃을 때,사랑한다 말하지 않아도 알아,널 만나러...",케이시 (Kassy),발라드,조영수
...,...,...,...,...,...,...,...
92550,26101,26146,음주운전 하지마 (하지마송part.1),"하지마 하지마 음주운전 하지마,하지마 하지마 음주운전 하지마,술먹고 운전했었어,음주...",강건,댄스,강건
92551,26101,26147,All I Want for Christmas Is You (Feat. Edward ...,"merry christmas,큰마음 먹고 널 위해 준비한 데이트,나 홀로 집에 있지...","커플다이어리, 아우라 (AOORA), $milli, 윤제","댄스, 인디음악",아우라 (AOORA)
92552,26101,26148,Starlight (inst.),"저 떨어지는 별에 빌었어,지금 네 맘도 나와 똑같기를,잠든 밤이 지나고 네가 떠오를...",ENOi,댄스,JYMON
92553,26101,26149,사막별 (inst.),"소중한 무언가를 난 또 쫓고 있어,어떤 모양인지조차,생각해 볼 순간도 없고,너에게만...",ENOi,댄스,페이퍼플래닛 (Paper Planet)


In [11]:
full = []
for i in df['lyric']:
  full.append(i)

In [19]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:100000]
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [20]:
custom_tokenizer = CustomTokenizer(Mecab())

In [21]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=1000)

In [25]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=5,
                 top_n_words=10,
                 calculate_probabilities=True,
                 verbose = True)

In [ ]:
topics, probs = model.fit_transform(full)

Batches:   0%|          | 0/2893 [00:00<?, ?it/s]

2022-04-20 06:11:15,091 - BERTopic - Transformed documents to Embeddings
2022-04-20 06:12:25,453 - BERTopic - Reduced dimensionality with UMAP


In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_distribution(probs[0])

In [6]:
for i in range(0, 5):
  print(i,'번째 토픽 :', model.get_topic(i))

In [7]:
# 명사
df['topic']

0        [일, 상상, 일상, 모습, 행동, 밤, 사랑, 종일, 면, 생각, 날, 행복, 정...
1        [집, 골목, 놀이터, 친구, 오랜만, 술, 얼굴, 맘, 사람, 생각, 대책, 진짜...
2             [날, 생각, 목소리, 노랫말, 미안, 후회, 미련, 진심, 말, 대답, 사랑]
3        [눈물, 길, 사람, 손길, 생각, 가슴, 사랑, 이별, 아이, 그리움, 바람, 옷...
4        [때, 사랑, 말, 길, 입가, 미소, 눈물, 밤하늘, 별, 보석, 힘, 옆, 사람...
                               ...                        
92550    [술, 운전, 음주, 잔, 차, 대리운전, 한참, 떡, 졸음운전, 눈, 정신, 상대...
92551    [마음, 준비, 데이트, 집, 약속, 명동, 거리, 눈, 행복, 말, 스타일, 취향...
92552    [별, 맘, 밤, 때, 기도, 새벽, 하늘, 문, 귓가, 날, 속삭임, 오늘, 시간...
92553    [무언가, 모양, 생각, 순간, 사막, 속, 별, 길, 끝, 방향, 새벽, 밤, 앞...
92554    [잔, 친구, 고생, 현실, 슬픔, 걱정, 사랑, 상처, 그리움, 시간, 돈, 김밥...
Name: topic, Length: 92555, dtype: object

In [8]:
df['genres_1'] = df['genres'].str.split(',',0).str[0]

In [9]:
ballad = df[df['genres_1'] == '발라드']
dance = df[df['genres_1'] == '댄스']

In [10]:
dance = dance.reset_index().drop(columns = 'index')
ballad = ballad.reset_index().drop(columns = 'index')

In [11]:
result_ballad = ballad['topic']
result_dance = dance['topic']

In [28]:
df['topic'].apply(lambda x: x.replace(', ',' '))

TypeError: ignored